In [ ]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import numpy as np
import pickle
from IPython.display import clear_output

from a_star_pathfinding import a_star
from vision import lines
from vision import transform

In [ ]:
def euclidean(pos_a, pos_b):
    return ((pos_a[0] - pos_b[0])**2 + (pos_a[1] - pos_b[1])**2)**.5

In [ ]:
class AmongUs(Model):
    def __init__(self, map_name, n_crew, n_impo, starting_positions, 
    num_tasks_crewmate=4, injob_time=(5,15), impostor_tactic='active', 
    impostor_behavior='aggressive', impostor_cooldown=214, impostor_vents=True,
    just_killed_cooldown=5, sus_kill=1, sus_vent=1, sus_task=.1, sus_group=.01,
    sus_default=.0005, n_iterated_games = 1):
        super().__init__()
        
        # changable parameters
        self.injob_time = injob_time
        self.num_tasks_crewmate = num_tasks_crewmate
        self.starting_positions = starting_positions
        self.impostor_tactic = impostor_tactic
        self.impostor_behavior = impostor_behavior
        self.impostor_cooldown = impostor_cooldown
        self.impostor_vents = impostor_vents
        self.just_killed_cooldown = just_killed_cooldown
        self.n_iterated_games = n_iterated_games
        
        # changable sus matrix parameters
        self.sus_kill = sus_kill
        self.sus_vent = sus_vent
        self.sus_task = sus_task
        self.sus_group = sus_group
        self.sus_default = sus_default
        
        # reset to 0 every game for indexing which agent is which in social matrices
        self.index_agent = 0
        
        # generate grid
        self.height = 138
        self.width = 242
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # generate map
        self.generate_map(map_name)
        
        # generate tasks
        self.generate_tasks(map_name)
        
        # load vision dict for impostors
        self.vision_dict = np.load('vision_dict.pkl', allow_pickle=True)
        
        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        self.schedule_Impostor = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.n_run = 0
        self.dead_players = []
        self.respawn_players()
        self.tasks_counter = 0
        
        self.crew_done = 0
        self.dead_crewmates = []
        
        # Generate sus matrix
        n_players = n_crew + n_impo
        self.sus_matrix = np.full((n_players, n_players), .5)
        self.trust_list = np.load(f'social_matrices/trust_{self.n_run}.npy', allow_pickle=True)
        
    def generate_map(self, map_name):
        hard_walls = np.load(f'{map_name}/hardwalls.npy')
        vents = np.load(f'{map_name}/vents.npy', allow_pickle=True)
        obstructions = np.load(f'{map_name}/obstructions.npy', allow_pickle=True)
        
        for coord in hard_walls:
            self.new_agent(Wall, tuple(coord))

        self.vents_dict = {}
        self.vents = []

        for connection in vents:
            connection = connection.tolist()
            self.vents_dict[tuple(connection[0])] = tuple(connection[1])
            self.vents_dict[tuple(connection[1])] = tuple(connection[0]) 
            
            if len(connection) == 3:
                self.vents_dict[tuple(connection[0])] = tuple(connection[2])
                self.vents_dict[tuple(connection[1])] = tuple(connection[2])
                self.vents_dict[tuple(connection[2])] = tuple(connection[0])
                self.vents_dict[tuple(connection[2])] = tuple(connection[1])
            
        for coord in self.vents_dict:
            self.new_agent(Vent, tuple(coord))
        
        for coord in obstructions:
            self.new_agent(Obstruction, tuple(coord))
    
    def generate_tasks(self, map_name):
        # load the image + coordinates of the short tasks and common tasks
        short_tasks = np.load(f'{map_name}/shorttasks.npy', allow_pickle=True)
        common_tasks = np.load(f'{map_name}/commontasks.npy', allow_pickle=True)
        
        self.short_tasks = short_tasks
        self.s_tasks = []
        
        self.common_tasks = common_tasks
        self.c_tasks = []

        self.all_tasks = []

        # load all the short_tasks into the map (all possible task locations), first create tuples
        for i in range(len(short_tasks)):
            if i == 3:
                self.new_agent(ShortTask, tuple(short_tasks[i][0][0]))
                for parttwo in range(8):
                    self.new_agent(ShortTask, tuple(short_tasks[i][1][parttwo]))
            elif i == 6:
                for partone in range(5):
                    self.new_agent(ShortTask, tuple(short_tasks[i][0][partone]))
                self.new_agent(ShortTask, tuple(short_tasks[i][1][0]))
        
            else:
                self.new_agent(ShortTask, tuple(short_tasks[i][0]))
        
        # Do the same for common tasks
        self.new_agent(CommonTask, tuple(common_tasks[0][0]))
        for i in range(len(common_tasks[1])):
            self.new_agent(CommonTask, tuple(common_tasks[1][i][0]))

    def respawn_players(self):
        for crewmate in self.schedule_Crewmate.agents:
            self.remove_agent(crewmate)
        for impostor in self.schedule_Impostor.agents:
            self.remove_agent(impostor)
            
        self.activated_agents = []
        self.dead_crewmates = []      
        self.crew_done = 0
        
        # Randomly distribute the index of the impostor
        impostor_index = random.randint(0, self.n_crew + self.n_impo - 1)
        for i in range(self.n_crew + self.n_impo):
            if i == impostor_index: 
                self.new_agent(Impostor, random.choice(self.starting_positions))
            else:
                self.new_agent(Crewmate, random.choice(self.starting_positions))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)

        if agent_type == Crewmate or agent_type == Impostor:
            getattr(self, f'schedule_{agent_type.__name__}').add(agent)
            agent.injob_time = self.injob_time
            agent.num_tasks = self.num_tasks_crewmate
            self.activated_agents.append(agent)
            self.index_agent += 1
            
        if agent_type == Dead_crewmate:
            self.dead_crewmates.append(agent)
            
        if agent_type == Vent:
            self.vents.append(agent)
        
        if agent_type == ShortTask:
            self.s_tasks.append(agent)
            self.all_tasks.append(agent)
        
        if agent_type == CommonTask:
            self.c_tasks.append(agent)
            self.all_tasks.append(agent)
            
    def remove_agent(self, agent):
        '''
        Method that removes an agent from the grid and the correct scheduler.
        '''
        self.grid.remove_agent(agent)
        getattr(self, f'schedule_{type(agent).__name__}').remove(agent)
        
        
    def vote_off(self):
        '''
        Method that determines which player gets voted off. 
        Player with highest trust*sus score gets voted off, if difference is significant with number two value
        '''
        # Sus matrix; row (R) is what player R thinks of players (C1, C2, C3, C4, C5)
        for player in self.dead_players:
            self.sus_matrix[player,:] = -np.inf
            self.sus_matrix[:,player] = -np.inf
            
        for i in range(len(self.sus_matrix)):
            self.sus_matrix[i, i] = -np.inf
            
        # Create vote_matrix (value between 0 and 1)
        vote_matrix = self.sus_matrix.copy()
        for i in range(len(vote_matrix)):
            for j in range(len(vote_matrix[i])):
                vote_matrix[i][j] = .5 + .5*np.tanh(vote_matrix[i][j])
  
        # Check trust matrix and sus matrux to determine who gets voted off
        trust_list = self.trust_list
        trust_score = trust_list.copy()
        total_scores = trust_score @ vote_matrix
        
        # Determine who is voted out
        voted_out = random.choice(np.argwhere(total_scores[0] == np.amax(total_scores[0])))[0]
        self.dead_players.append(voted_out)
        
        # remove agent form game and activated agent list
        self.remove_agent(self.activated_agents[voted_out])
        
        # finish task of voted out agent
        if type(self.activated_agents[voted_out]) == Crewmate:
            if not self.activated_agents[voted_out].done:
                self.crew_done += 1
                self.activated_agents[voted_out].done = True
        
        # Change trust matrix depenending on correct choices, trust increases for correct sus, decreases for wrong sus (max with factor 0.05)
        for player_trust in range(len(self.trust_list[0])):
            
            vote_list = vote_matrix[player_trust]
            # Determine who the player voted for
            player_vote = np.argmax(vote_list, axis = 0)
            
             # Check if the vote was correct, lower or increase trust value accordingly
            if type(self.activated_agents[player_vote]) == Impostor:
                self.trust_list[0][player_trust] += .06
                round(self.trust_list[0][player_trust], 3)
                
                if self.trust_list[0][player_trust] > 1:
                    self.trust_list[0][player_trust] = 1
                
            if type(self.activated_agents[player_vote]) == Crewmate:
                self.trust_list[0][player_trust] -= .02
                round(self.trust_list[0][player_trust], 3)
                
                if self.trust_list[0][player_trust] < 0:
                    self.trust_list[0][player_trust] = 0
        
        # set players to base
        for i in range(len(self.activated_agents)):
            if i not in self.dead_players:
                self.activated_agents[i].path = [(130, 100)]
                for impostor in self.schedule_Impostor.agents:
                    impostor.cooldown = self.impostor_cooldown
                    impostor.just_killed = 0
        
        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()
        self.schedule_Impostor.step()

        # update sus matrix every step
        self.sus_matrix += self.sus_default
        

    def play_match(self, iterations=0, match_num=0):
        '''
        Method that runs a single match.
        '''
        #reset to 0 every game for indexing which agent is which in social matrices
        self.index_agent = 0
        self.dead_players = []
        self.crew_done = 0
        self.tasks_counter = 0
        
        n_players = self.n_crew + self.n_impo
        self.sus_matrix = np.full((n_players, n_players), .5)
        #make sure the trust matrix is reset after 'new players' enter a game
        if self.n_run % self.n_iterated_games == 0:
            self.trust_list = np.load(f'social_matrices/trust_0.npy', allow_pickle=True)
        
        self.respawn_players()
        i = 1
        while len(self.schedule_Crewmate.agents) > self.n_impo and len(self.schedule_Impostor.agents) > 0 and self.crew_done != self.n_crew:
            print(f'Match: {match_num+1}/{iterations}')
            print(f' Step: {i}')
            clear_output(wait=True)
            i += 1
            self.step()
        
        print(len(self.schedule_Crewmate.agents) > self.n_impo)
        print(len(self.schedule_Impostor.agents) > 0)
        print(self.crew_done != self.n_crew)
        print(f'Ended at iteration: {i}')
        print(f'Number of tasks completed: {self.tasks_counter}')
        print(f'Number of crewmates finished: {self.crew_done}')
        #save the trust matrix to be loaded later
        self.n_run += 1
        np.save(f'social_matrices/trust_{self.n_run}.npy', self.trust_list)
        
        if i < 200:
            raise
        # NEW
        # load previous wins
        self.win_matrix = np.load(f'win_matrices/win_matrix_{self.n_run - 1}.npy', allow_pickle=True)
      
        # crewmates win
        if len(self.schedule_Impostor.agents) == 0 or self.crew_done == self.n_crew:
            print('The crewmates won!')
            for player in self.activated_agents:
                if type(player) != Impostor:
                    self.win_matrix[player.index, 0] += 1
        
        # impostor wins
        else:
            print('The impostor won!')
            impostor = self.schedule_Impostor.agents[0]
            self.win_matrix[impostor.index, 1] += 1
        
        # add wins
        np.save(f'win_matrices/win_matrix_{self.n_run}.npy', self.win_matrix)
        
        
        # NEW save thins
        iteration_data = np.load(f'data_1/iteration_data.npy', allow_pickle=True)
        iteration_data = np.append(iteration_data, i)
        np.save(f'data_1/iteration_data.npy', iteration_data)

        tasks_data = np.load(f'data_1/tasks_data.npy', allow_pickle=True)
        tasks_data = np.append(tasks_data, self.tasks_counter)
        np.save(f'data_1/tasks_data.npy', tasks_data)

        crewmates_done_data = np.load(f'data_1/crewmates_done_data.npy', allow_pickle=True)
        crewmates_done_data = np.append(crewmates_done_data, self.crew_done)
        np.save(f'data_1/crewmates_done_data.npy', crewmates_done_data)

        win_data = np.load(f'data_1/win_data.npy', allow_pickle=True)
        if len(self.schedule_Impostor.agents) == 0 or self.crew_done == self.n_crew:
            win_data = np.append(win_data, 0) 
        else:
            win_data = np.append(win_data, 1)
        np.save(f'data_1/win_data.npy', win_data)

        dead_data = np.load(f'data_1/dead_data.npy', allow_pickle=True)
        dead_data = np.append(dead_data, len(self.dead_players))
        np.save(f'data_1/dead_data.npy', dead_data)
        
    def run(self, iterations):
        '''
        Method that runs multiple matches.
        '''
        for match_num in range(iterations):
            self.play_match(iterations, match_num)
            clear_output(wait=True)

In [ ]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goallist = []
        self.injob = 0
        self.num_tasks = self.model.num_tasks_crewmate
        self.injob_time = self.model.injob_time
        # self.create_dict()
        self.index = self.model.index_agent
        self.make_tasks()
        self.done = False
        
    def make_tasks(self):
        short_tasks = self.model.short_tasks
        common_tasks = self.model.common_tasks
        
        # set 4 random tasks for Crewmate to execute (can still be same task multiple times)
        self.goallist = random.sample(list(short_tasks), self.num_tasks)
        
        # Import the common tasks
        self.goallist.append(common_tasks[0])
        self.fix_wires_tasks = [random.choice(common_tasks[1])]
        
        self.goallist.append([self.fix_wires_tasks[0][0]])
        
        # if it is a task with multiple locations/second part, replace by one of the coordinates from the first list 
        self.goallist = [[random.choice(self.goallist[index][0])] if len(x)>1 else x for index,x in enumerate(self.goallist)]
        
        # Turn list into list of tuples
        self.goallist = [tuple(self.goallist[index][0]) for index,y in enumerate(self.goallist)]
        
    def find_path(self):
              
        # Find shortest path and go to that task
        length1 = np.inf
        
        for t in self.goallist:
            path, length2 = a_star(self.pos, t, self.model.grid)
            
            if length2 < length1:
                length1 = length2
                shortest_path = path

        self.path = self.path + shortest_path
        
        
    def detect_agents(self):
        
        # Create empty list for agents
        agents_detected = []
        visible_area = self.model.vision_dict[self.pos]
        
        # Check all the vissible squares
        for pos in visible_area:
            agents_on_tile = self.model.grid.get_neighbors(pos=pos, moore=True, radius=0, include_center=True)
            
            # If another player, append it to the detected list
            for agent in agents_on_tile:
                if type(agent) == Crewmate or type(agent) == Impostor:
                    agents_detected.append(agent)
                
                # If dead agent found, start voting prodecure
                if type(agent) == Dead_crewmate:
                    for dead_crewmate in self.model.dead_crewmates:
                        self.model.grid.remove_agent(dead_crewmate)
                        self.model.dead_crewmates.remove(dead_crewmate)
                    self.model.vote_off()
                    
        return agents_detected
    
    def die(self):
        self.model.new_agent(Dead_crewmate, self.pos)
        #automatically finishes tasks
        if not self.done:
            self.model.crew_done += 1
            self.model.tasks_counter += len(self.goallist)
            self.done = True
        self.model.remove_agent(self)
        # change sus sccore of dead agent to 0
        self.model.dead_players.append(self.index)

    def step(self):
        if self.path == []:
            if self.pos in self.goallist:
                self.goallist.remove(self.pos)
                self.injob = random.randint(self.injob_time[0], self.injob_time[1])
                self.model.tasks_counter += 1
                
            # if Task was task 3, add second part of task 3 to the task list
            if self.pos == tuple(self.model.short_tasks[3][0][0]):
                self.goallist.append(tuple(random.choice(self.model.short_tasks[3][1])))
            
            # if Task was task 6, add second part of task 6 to the task list
            elif self.pos in self.model.short_tasks[6][0]:
                self.goallist.append(tuple(self.model.short_tasks[6][1][0]))
                
            # if Task was common task Fix Wires, append next one
            elif self.pos in np.array(self.fix_wires_tasks):
                for i in range(2):
                    if self.pos == tuple(self.fix_wires_tasks[0][i]):
                        self.goallist.append(tuple(self.fix_wires_tasks[0][i+1]))
                        
            # if all tasks are done, go to middle and run around
            if len(self.goallist) == 0:
                self.goallist = [(130, 100), (121, 107), (139, 108),(130, 114)]
                if not self.done:
                    self.model.crew_done += 1
                    self.done = True
                
            self.find_path()

        elif self.path != []:
            # if crewmate is on cooldown for doing a job stand still
            if self.injob > 0:
                self.injob -= 1
            else:
                self.model.grid.move_agent(self, self.path.pop()) 

        # find and loop over all detected impostors and crewmates in vision radius
        task_list = [(101, 55), (237,83), (167,85), (83, 58), (170,18), 
                     (197,42), (239,75), (7, 88), (149, 63), (158, 18), 
                     (78, 58), (177, 20), (168, 52), (125, 50), (91, 55), 
                     (140, 63), (47, 73), (220, 77), (106, 132)]
        agents_detected = self.detect_agents()
        for agent in agents_detected:
            if type(agent) == Impostor and agent.just_killed > 0:
                self.model.sus_matrix[self.index, agent.index] += self.model.sus_kill
                
            if agent.pos in self.model.vents:
                self.model.sus_matrix[self.index, agent.index] += self.model.sus_vent
                
            if agent.pos in task_list:
                self.model.sus_matrix[self.index, agent.index] += self.model.sus_task
            
            else:
                self.model.sus_matrix[self.index, agent.index] += self.model.sus_group
            
class Impostor(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.vents_enabled = self.model.impostor_vents
        # tactic can be active or passive
        self.tactic = self.model.impostor_tactic
        # behavior can be carelful or agressive
        self.behavior = self.model.impostor_behavior
        self.cooldown = self.model.impostor_cooldown
        self.just_killed = 0
        self.index = self.model.index_agent
        # impostor needs initial goal
        self.goals = [random.choice(self.model.all_tasks).pos]
        self.find_path()

    def find_path(self):
        if len(self.goals) == 0:
            # select random task as goal
            self.goals = [random.choice(self.model.all_tasks).pos]
        
        else:
            goal = self.goals.pop()
            best_path, best_score = a_star(self.pos, goal, self.model.grid)

            if self.vents_enabled:
                for vent in self.model.vents:
                    vent_in = vent.pos_a
                    vent_out = vent.pos_b
                    guess_vent_path = euclidean(self.pos, vent_in) + euclidean(vent_out, goal)

                    if guess_vent_path < best_score:
                        vent_path_in, vent_score_in = a_star(self.pos, vent_in, self.model.grid)
                        vent_path_out, vent_score_out = a_star(vent_out, goal, self.model.grid)
                        vent_path = vent_path_out + vent_path_in
                        vent_score = vent_score_in + vent_score_out
    
                        if vent_score < best_score:
                            best_score = vent_score
                            best_path = vent_path

            self.path = best_path

    def detect_crewmates(self):
        crewmates_detected = []
        visible_area = self.model.vision_dict[self.pos]
        for pos in visible_area:
            agents_on_tile = self.model.grid.get_neighbors(pos=pos, moore=True, radius=0, include_center=True)
            for agent in agents_on_tile:
                if type(agent) == Crewmate:
                    crewmates_detected.append(agent)
        return crewmates_detected

    def step(self):
        if self.cooldown > 0:
            self.cooldown -= 1
            
        if self.just_killed > 0:
            self.just_killed -= 1
            
        if self.path == []:
            if self.tactic == 'active':
                self.find_path()
            elif self.tactic == 'passive' and self.just_killed > 0:
                self.find_path()

        crewmates_detected = self.detect_crewmates()

        for crewmate in crewmates_detected:
            if self.cooldown == 0 and euclidean(crewmate.pos, self.pos) <= 6:
                # kill a crewmate immediately
                if self.behavior == 'aggressive':
                    crewmate.die()
                    self.cooldown = self.model.impostor_cooldown
                    self.just_killed = self.model.just_killed_cooldown

                # wait a couple of steps before killing
                elif self.behavior == 'careful':
                    self.waiting_countdown = 5
                    self.behavior = 'waiting'
                elif self.behavior == 'waiting':
                    if self.waiting_countdown > 0:
                        self.waiting_countdown -= 1
                    else:
                        # only kill if there is 1 crewmate in sight
                        if len(self.detect_crewmates()) == 1:
                            crewmate.die()
                            self.cooldown = self.model.impostor_cooldown
                            self.just_killed = self.model.just_killed_cooldown
                            self.behavior = 'careful'
                        else:
                            #deze self.cooldown moet weg vgm want er is niet gekilled
                            #self.cooldown = self.model.impostor_cooldown
                            self.behavior = 'careful'
                
        if self.path != []:
            self.model.grid.move_agent(self, self.path.pop())              
class Dead_crewmate(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos
    
class Wall(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Obstruction(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Vent(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos_a = pos
        self.pos_b = self.model.vents_dict[pos]

class ShortTask(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos
        
class CommonTask(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

In [ ]:
# initialize first trust_list
np.save('social_matrices/trust_0.npy', np.full((1, 6), .5))
np.save('win_matrices/win_matrix_0.npy', np.full((6,2), 0))

# fixed_parameters
starting_positions = [(130, 100), (121, 107), (139, 108), 
(130, 114), (123, 103), (137, 103), (136,112), (124,112)]

# varied parameters
number_of_crewmates = 5
num_tasks_crewmate = 4
injob_time = (5, 15)
impostor_tactic = 'active' # active/passive
impostor_behavior = 'aggressive' # aggressive/careful
impostor_cooldown = 214 # in-game value == 214
impostor_vents = True
just_killed_cooldown = 5
n_iterated_games = 1

# sus matrix parameters
sus_kill = np.inf
sus_vent = np.inf
sus_task = -.1
sus_group = -.01
sus_default = .0005

# NEW save things
# np.save(f'data/iteration_data.npy', [])
# np.save(f'data/tasks_data.npy', [])
# np.save(f'data/crewmates_done_data.npy', [])
# np.save(f'data/win_data.npy', [])
# np.save(f'data/dead_data.npy', [])

amongUs = AmongUs('the_skeld', number_of_crewmates, 1, starting_positions, 
num_tasks_crewmate=num_tasks_crewmate, injob_time=injob_time, impostor_tactic=impostor_tactic,
impostor_behavior=impostor_behavior, impostor_cooldown=impostor_cooldown, impostor_vents=impostor_vents,
just_killed_cooldown=just_killed_cooldown, sus_kill=sus_kill, sus_vent=sus_vent, sus_task=sus_task,
sus_group=sus_group, sus_default=sus_default, n_iterated_games=n_iterated_games)

amongUs.run(10)